<a href="https://colab.research.google.com/github/5j54d93/CNNs-fruits360/blob/main/CNN's%20neural%20architecture%20include%20ResBlock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Fruits360 data set

In [ ]:
import tensorflow as tf

url = 'https://github.com/Horea94/Fruit-Images-Dataset/archive/master.zip'
name = 'fruits360'
file = tf.keras.utils.get_file(name, url, extract=True)

In [ ]:
import os

# 指定訓練資料夾目錄
training_path = os.path.sep.join(file.split(os.path.sep)[0:-1]+['Fruit-Images-Dataset-master','Training'])

# 指定測試資料夾目錄
test_path     = os.path.sep.join(file.split(os.path.sep)[0:-1]+['Fruit-Images-Dataset-master','Test'])

# 定義訓練影像資料產生器
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.1)
# 定義測試影像資料產生器
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# 訓練資料產生器
train_generator = train_datagen.flow_from_directory(training_path,target_size=(100,100),batch_size=32,class_mode='sparse',subset='training')
# 驗證資料產生器
valid_generator = train_datagen.flow_from_directory(training_path,target_size=(100,100),batch_size=32,class_mode='sparse',subset='validation')
# 測試資料產生器
test_generator  = test_datagen.flow_from_directory(test_path,target_size=(100,100),batch_size=32,class_mode='sparse')

In [ ]:
#類別名稱與代號
print(list(train_generator.class_indices.keys())[1])

#訓練樣本數量與類別數量
print(train_generator.n, train_generator.num_classes)

# 顯示部分訓練資料

In [ ]:
import matplotlib.pyplot as plt

indices_class = {v:name for name,v in train_generator.class_indices.items()}

(X,y)=train_generator.next()

plt.figure(figsize=(16,8))
for i in range(X.shape[0]):
    plt.subplot(4,8,i+1)
    plt.imshow(X[i])
    plt.title(indices_class[int(y[i])])
    plt.axis(False)
plt.tight_layout()    
plt.show()        

# 顯示部分驗證資料

In [ ]:
(X,y)=valid_generator.next()
plt.figure(figsize=(16,8))
for i in range(X.shape[0]):
    plt.subplot(4,8,i+1)
    plt.imshow(X[i])
    plt.title(indices_class[int(y[i])])
    plt.axis(False)
plt.tight_layout()    
plt.show()

# 顯示部分測試資料

In [ ]:
(X,y)=test_generator.next()
plt.figure(figsize=(16,8))
for i in range(X.shape[0]):
    plt.subplot(4,8,i+1)
    plt.imshow(X[i])
    plt.title(indices_class[int(y[i])])
    plt.axis(False)
plt.tight_layout()    
plt.show()     

# 定義卷積神經網路架構

In [ ]:
images = keras.layers.Input(x_train.shape[1:])

x = keras.layers.Conv2D(filters=16, kernel_size=[1,1], padding='same')(images)
block = keras.layers.Conv2D(filters=16, kernel_size=[3,3], padding="same")(x)
block = keras.layers.BatchNormalization()(block)
block = keras.layers.Activation("relu")(block)
block = keras.layers.Conv2D(filters=16, kernel_size=[3,3],padding="same")(block)
net = keras.layers.add([x,block])
net = keras.layers.BatchNormalization()(net)
net = keras.layers.Activation("relu")(net)
net = keras.layers.MaxPooling2D(pool_size=(2,2),name="block_1")(net)
x = keras.layers.Conv2D(filters=32, kernel_size=[1,1], padding='same')(net)
block = keras.layers.Conv2D(filters=32, kernel_size=[3,3], padding="same")(x)
block = keras.layers.BatchNormalization()(block)
block = keras.layers.Activation("relu")(block)
block = keras.layers.Conv2D(filters=32, kernel_size=[3,3],padding="same")(block)
net = keras.layers.add([x,block])net=keras.layers.BatchNormalization()(net)
net = keras.layers.Activation("relu")(net)
net = keras.layers.MaxPooling2D(pool_size=(2,2),name="block_2")(net)

x = keras.layers.Conv2D(filters=64, kernel_size=[1,1], padding='same')(net)
block = keras.layers.Conv2D(filters=64, kernel_size=[3,3], padding="same")(x)
block = keras.layers.BatchNormalization()(block)
block = keras.layers.Activation("relu")(block)
block = keras.layers.Conv2D(filters=64, kernel_size=[3,3],padding="same")(block)
net = keras.layers.add([x,block])
net = keras.layers.Activation("relu", name="block_3")(net)

net = keras.layers.BatchNormalization()(net)
net = keras.layers.Dropout(0.25)(net)

net = keras.layers.GlobalAveragePooling2D()(net)
net = keras.layers.Dense(units=nclasses,activation="softmax")(net)

model = keras.models.Model(inputs=images,outputs=net)

model.summary() 

# 訓練卷積神經網路

In [ ]:
history = model.fit(train_generator,epochs=10,validation_data=valid_generator)

# 顯示訓練損失歷史曲線

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'],label='accuracy')
plt.plot(history.history['val_accuracy'],label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# 顯示部分測試結果

In [ ]:
import numpy as np
(X,y)=test_generator.next()
py   = model.predict(X)
plt.figure(figsize=(16,32))
for i in range(X.shape[0]):
    plt.subplot(8,4,i+1)
    plt.imshow(X[i])
    plt.title(indices_class[np.argmax(py[i])]+'/'+indices_class[int(y[i])])
    plt.axis(False)
plt.tight_layout()    
plt.show()   